# EDA for ENSEMBLE LEARNING ASSESMENT

In [1]:
import pandas as pd

### Lectura de Datos

In [3]:
nom_fi_datos_Irrad = "data/G10A_DATOS_IRRAD.csv"
df_orig_Irrad = pd.read_csv(nom_fi_datos_Irrad)

# parseamos la fecha (cadena) para que sea un datetime con formato yyyy/mm/dd

df_orig_Irrad['FECHA'] = pd.to_datetime(df_orig_Irrad['FECHA'], format='%Y-%m-%d')

s = df_orig_Irrad.loc[:,'FECHA']
df_orig_Irrad['FECHA'] =  s.dt.date

In [8]:
nom_fi_datos_Util = "data/G10A_DATOS_UTIL.csv"
df_orig_Util = pd.read_csv(nom_fi_datos_Util)

df_orig_Util['FECHA'] = pd.to_datetime(df_orig_Util['FECHA'], format='%Y-%m-%d')

s = df_orig_Util.loc[:,'FECHA']
df_orig_Util['FECHA'] =  s.dt.date

El primero de los dos dataframes contiene datos de irradiación solar acumulada cada 3 horas (es decir, la irradaición solar del tramo de la hora 0 es la acumulada de las horas 0, 1 y 2, su suma). Se conoce como irradiación solar a la irradiación neta absorbida por la superficie terrestre, siendo ésta la suma de la radiación directa, difusa (a través de nubes) y reflejada (por ejemplo, reflejada del suelo).

Por otro lado tenemos los registros correspondientes de la utilización solar fotovoltaica, que se define como la generación solar horaria entre la potencia (por lo que es adimensional, y se encuentra entre 0 y 1). Además, la utilización horaria deñ tramo h no es la suma de las horas $h$, $h+1$ y $h+2$, sino su media.